In [ ]:
import math

class Node:
    def __init__(self, x, y, is_cotton_plant=False, has_weed=False):
        self.x = x
        self.y = y
        self.is_cotton_plant = is_cotton_plant
        self.has_weed = has_weed
        self.g_score = float('inf')
        self.f_score = float('inf')
        self.parent = None

    def __repr__(self):
        return f'Node({self.x}, {self.y})'

In [ ]:
def create_grid(width, height, cotton_spacing):
    grid = [[0] * width for _ in range(height)]
    ### Note i for width and j for height
    for i in range(0, width, cotton_spacing):
        for j in range(0, height, cotton_spacing):
            grid[i][j] = 1
    return grid

grid = create_grid(11, 11, 2)
grid

[[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]]

In [ ]:
grid[10][9]  ### starting point

0

In [ ]:
from IPython.core.display import Javascript
class Robot:
    def __init__(self, grid, cotton_plants, start, goal):
        self.grid = grid
        self.width = len(grid[0])
        self.height = len(grid)
        self.cotton_plants = cotton_plants
        self.start = start
        self.goal = goal

    def check_closed_list(self, x, y, closed_list):
        for i in closed_list:
            if i.x == x and i.y == y:
                return False
        return True

    def get_neighbors(self, node, closed_list):
        neighbors = []
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            x, y = node.x + dx, node.y + dy
            if 0 <= x < self.width and 0 <= y < self.height:  ## sanity check whether the neighbours are in the defined grid
                if self.check_closed_list(x, y, closed_list):    ## The neighbour should not be the visited node
                    if self.grid[y][x] != 1:   ### this ignores all the neighbours with the cotton plant
                        neighbors.append(Node(x, y))
        return neighbors

    def cost(self, node1, node2):    ### this cost calculates distance from the start node to the current node.
        #print(math.sqrt((node1.x - node2.x)**2 + (node1.y - node2.y)**2))
        return math.sqrt((node1.x - node2.x)**2 + (node1.y - node2.y)**2)

    def heuristic(self, node1, node2):      ### this cost will calculate the perpendicular distance of the node from line joining actual start and goal node
        return 10*abs(node1.y - node2.y)  #node2: goal node and node1: current node

    def a_star(self, start_node, goal_node):
        #start_node = Node(*self.start)
        start_node.g_score = 0
        start_node.f_score = self.heuristic(start_node, goal_node)
        open_list = [start_node]
        closed_list = []

        i = 0
        while open_list:
            # print("i:", i)
            # print("current node:", min(open_list, key=lambda x: x.f_score))
            # for j in open_list:
            #     print("Node:", j, "g_score", j.g_score)
            #     print("Node:", j, "f_score", j.f_score)
            current = min(open_list, key=lambda x: x.f_score)   #finds the node with minimum f_score in the open_list

            if current.x == goal_node.x and current.y == goal_node.y:  #if the current node reaches the goal_node, that means the path has been generated
                return self.reconstruct_path(current)   #### GOAL NODE REACHED ####

            # print("Current Node:", current)
            open_list.remove(current)
            closed_list.append(current)
            # print("Open List:", open_list)
            # print("Closed List:", closed_list)

            for neighbor in self.get_neighbors(current, closed_list):
                tentative_g_score = current.g_score + self.cost(current, neighbor) + self.cost(neighbor, self.goal)  #tentative g_score of the neighbour
                #print("Neighbor:", neighbor, "g_score", tentative_g_score)
                if tentative_g_score < neighbor.g_score:
                    neighbor.parent = current
                    neighbor.g_score = tentative_g_score
                    neighbor.f_score = neighbor.g_score + self.heuristic(neighbor, goal_node)
                    if neighbor not in open_list:
                        open_list.append(neighbor)
            # if i==10:
            #   break
            i+=1


        return None

    def reconstruct_path(self, node):   ### Drawing the entire path from the current node to all its parent nodes.
        path = [node]
        while node.parent:
            path.append(node.parent)
            node = node.parent
        # print(path)
        return path[::-1]

    def update_grid(self, node):
        self.grid[node.y][node.x] = 0

    def run(self):

        start_node = self.start
        # if start_node.x!= 0:
        #   goal_node = Node(0, start_node.y)     #added line
        # else:
        #   goal_node = Node(start_node.x, self.height-1)

        print("Start_Node:", self.start)
        print("Goal_Node:", self.goal)

        path = self.a_star(self.start, self.goal)
        print(path)


        # while (self.goal != start_node):
        # # Initialize the path
        #     path = self.a_star(start_node, goal_node)
        #     print("path")
        #     # Move the robot along the path
        #     for i in range(len(path) - 1):
        #         current_node = path[i]
        #         next_node = path[i+1]

        #         # Check if there are obstacles in the way
        #         if self.check_obstacles(current_node, next_node):  ########## USE THE ULTRASONIC DISTANCE SENSORS FOR THIS TASK #########
        #             print("Obstacle detected, replanning path...")
        #             ########### Change the route of the robot using predefined movement for obstacle route change,
        #             ########### and change the current node to new node on which the robot has moved to

        #             new_path = self.a_star(start=(current_node, goal_node))
        #             if new_path:
        #                 path = path[:i] + new_path
        #             else:
        #                 print("No path found, stopping robot.")
        #                 return

                # # Move the robot to the next node    ######### Define the move_to function that takes the current node and accordingly moves the robot to the next node.
                # self.move_to(current_node, next_node)

                # # Detect weed using the camera and update the nect_node's has weed status accordingly.

                # # Cut any weeds at the current node
                # if current_node.has_weed:
                #     self.cut_weed(current_node)

            # new_start_node = goal_node  #New start node becomes the previous goal node
            # goal_node = start_node.x - 2, start_node.y  #The new goal node will be 2 rows to the left of the previous start node
            # start_node = new_start_node
            # print(path)


        # Print a message when the task is completed
        print("Task completed, all weeds have been cut!")


In [ ]:
grid

[[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]]

In [ ]:
import numpy as np
cotton_plants = np.where(np.array(grid) > 0)
cotton_plants = np.array(cotton_plants).transpose().tolist()

start  = Node(0, 9)
goal = Node(10, 7)

# Define the robot
robot = Robot(grid, cotton_plants, start, goal)

# Run the robot
robot.run()

Start_Node: Node(0, 9)
Goal_Node: Node(10, 7)
[Node(0, 9), Node(1, 9), Node(1, 8), Node(1, 7), Node(2, 7), Node(3, 7), Node(4, 7), Node(5, 7), Node(6, 7), Node(7, 7), Node(8, 7), Node(9, 7), Node(10, 7)]
Task completed, all weeds have been cut!
